# Automatic HE Parameter Selection

This notebook demonstrates how to use `tf-shell` to automatically choose low
level parameters (like the plaintext modulus and ciphertext moduli) for the BGV
HE scheme. While these parameters can be chosen manually, as shown in other
examples, it is convenient to let `tf-shell` choose them.

Since the HE parameters depend on the depth of a computation, they must be
fixed before the computation starts. `tf-shell` does this by extending
TensorFlow's graph compiler, grappler,  with some convenient
homomorphic-encryption (HE) specific features, one of which is automatic
parameter selection.

As such, automatic parameter selection is only available when using TensorFlow's
deferred execution mode (graph mode). This way, the graph is available for
inspection (to estimate ciphertext noise growth) and modification (to inject
generated parameters) before it is executed.

In [1]:
import tensorflow as tf
import tf_shell

2024-10-29 19:56:56.261116: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 19:56:56.287144: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
a = [1, 2, 3]
b = [4, 5, 6]

Here we define the function we'd like to compute. TensorFlow will first trace
this function (without executing it) to build a graph of the computation. Then,
during a graph compiler optimization pass, `tf-shell` will replace the
"autocontext" placeholder Op with parameters generated for this specific
computation based on statistical estimation of the noise growth and the initial
plaintext size.

Note, the `create_autocontext64` function must be called from inside a
`tf.function` in order to execute in deferred mode.

In [3]:
@tf.function
def foo(cleartext_a, cleartext_b):
    shell_context = tf_shell.create_autocontext64(
        log2_cleartext_sz=4,  # Maximum size of the cleartexts (including the scaling factor).
        scaling_factor=1,  # The scaling factor (analagous to fixed-point but not necessarily base 2).
        noise_offset_log2=0,  # Extra buffer for noise growth.
    )
    key = tf_shell.create_key64(shell_context)
    a = tf_shell.to_encrypted(cleartext_a, key, shell_context)
    b = tf_shell.to_shell_plaintext(cleartext_b, shell_context)

    intermediate = a * b
    result = tf_shell.to_tensorflow(intermediate, key)
    return result

In [4]:
tf_shell.enable_optimization()  # Enable the autoparameter graph optimization pass.

a = [1, 2, 3]
b = [4, 5, 6]
c = foo(a, b)

print(c)

Selected BGV parameters:
log_n: 11
t: 12289 (14 bits, min:4)
qs: 70371484213249  (47 bits, min:47 = t:14 + noise:33 + offset:0)
INFO: Generating key
tf.Tensor([ 4 10 18 ...  0  0  0], shape=(2048,), dtype=int32)


`tf-shell` selected a plaintext modulus `t` to be at least 4 bits, and ciphertext
modulus `Q` as a produt of smaller moduli `qs` for representation of ciphertexts
in RNS (residual number system). `Q` is chosen to be large enough to support
noise growth in the computation without overflowing. Since this computation is
small, only one ciphertext modulus is needed.

Note that `tf-shell` treats the first dimension of data as the packing dimension
of the BGV scheme (the slotting dimension). When the function is first traced,
the size of this dimension is unknown, because the ring degree of the
ciphertexts has not been chosen yet as it depends on `Q`, which depends on the
estimated noise growth. In the example above, the three elements of the input
vectors are packed into this first dimension for efficiency purposes. The
remaining slots in the ciphertexts went unused.